# CONFLUENCE Tutorial - 3: Lumped Basin Workflow (Bow River at Banff)

## Introduction

This tutorial shows how to scale up from point-scale modeling to basin-scale streamflow simulation using CONFLUENCE. Building on our previous tutorials with SNOTEL and FLUXNET data, we now demonstrate how to model an entire watershed as a single unit to generate streamflow at the basin outlet.

### What is Lumped Basin Modeling?

Lumped basin modeling treats the entire watershed as one homogeneous unit, averaging all the spatial variability across the catchment. While this is a simplification, it's a valuable approach because:

- **Simplicity**: Easier to understand and implement than distributed models
- **Computational efficiency**: Fast execution makes it ideal for calibration and uncertainty analysis  
- **Baseline performance**: Establishes whether a model can capture the basic watershed response before adding spatial complexity
- **Parameter identification**: Simpler structure makes it easier to understand which parameters control model behavior

### Case Study: Bow River at Banff

We'll use the Bow River at Banff as our example watershed:

- **Location**: Canadian Rockies, Alberta, Canada
- **Drainage area**: ~2,210 km²
- **Elevation**: Ranges from 1,384 m at the outlet to over 3,400 m in the headwaters
- **Climate**: Snow-dominated mountain system with pronounced seasonal cycles
- **Gauging station**: Water Survey of Canada station 05BB001 with long-term observations

This watershed presents interesting modeling challenges:
- Strong elevation gradients affecting temperature and precipitation
- Complex snow dynamics across elevation zones
- Seasonal storage in snowpack and glaciers
- Pronounced spring freshet from snowmelt

### What You'll Learn

This tutorial will teach you how to:

1. **Set up a basin-scale project** with CONFLUENCE's automated workflow
2. **Delineate watersheds** automatically from digital elevation models
3. **Aggregate spatial data** to create catchment-averaged characteristics
4. **Process meteorological forcing** data for basin-scale modeling
5. **Configure and run SUMMA** for lumped basin simulation
6. **Evaluate model performance** using standard hydrological metrics
7. **Interpret results** and understand model limitations

### Tutorial Overview

We'll walk through the complete CONFLUENCE workflow step by step:

1. **Project Setup**: Create the organized directory structure
2. **Watershed Delineation**: Automatically identify the watershed boundary
3. **Data Acquisition**: Get elevation, soil, and land cover data
4. **Forcing Data**: Process meteorological inputs
5. **Model Configuration**: Set up SUMMA for the lumped basin
6. **Model Execution**: Run the simulation
7. **Results Analysis**: Compare simulated and observed streamflow

By the end of this tutorial, you'll understand how CONFLUENCE handles the transition from point-scale to basin-scale modeling and be ready to explore more complex distributed modeling approaches.

## Step 1: Rapid Basin-Scale Workflow Setup
Building on the point-scale modeling expertise from Tutorials 01a and 01b, we now advance to basin-scale hydrological modeling. This represents a fundamental scaling transition: from process validation at individual sites to integrated watershed simulation that captures the collective hydrological response of an entire catchment.
Scaling Transition: Point → Basin

- Spatial Scale: Single location → Entire watershed (~2,210 km²)
- Process Integration: Isolated vertical processes → Integrated water balance with routing
- Validation Target: Local states (SWE, SM, LE) → Streamflow at basin outlet
- Complexity: Uniform characteristics → Spatially-averaged catchment properties
- Scientific Challenge: Process understanding → Emergent watershed behavior

The same CONFLUENCE architecture seamlessly handles this transition, demonstrating the framework's scalability from point validation through basin-scale prediction while maintaining reproducible workflow principles.

In [ ]:
# =============================================================================
# STEP 1: RAPID BASIN-SCALE WORKFLOW SETUP
# =============================================================================

# Import required libraries
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from datetime import datetime
import xarray as xr
import numpy as np

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import main CONFLUENCE class
from CONFLUENCE import CONFLUENCE

# Set up plotting style
plt.style.use('default')
%matplotlib inline

print("=== CONFLUENCE Tutorial 02a: Lumped Basin Modeling ===")
print("Scaling from point-scale validation to basin-scale streamflow simulation")

# =============================================================================
# CONFIGURATION FOR BOW RIVER AT BANFF WATERSHED
# =============================================================================

print("\n🏔️ Configuring for Bow River at Banff Watershed")

# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/Users/darrieythorsson/compHydro/data/CONFLUENCE_data')  # ← Update this path

# Load template configuration and customize for basin-scale modeling
config_template_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_template.yaml'

with open(config_template_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Update for Bow River basin-scale modeling
config_updates = {
    'CONFLUENCE_CODE_DIR': str(CONFLUENCE_CODE_DIR),
    'CONFLUENCE_DATA_DIR': str(CONFLUENCE_DATA_DIR),
    'DOMAIN_NAME': 'Bow_at_Banff',
    'EXPERIMENT_ID': 'lumped_basin_tutorial',
    'POUR_POINT_COORDS': '51.1722/-115.5717',  # Banff gauging station
    'DOMAIN_DEFINITION_METHOD': 'delineate',    # Watershed delineation vs point buffer
    'DOMAIN_DISCRETIZATION': 'lumped',          # Single HRU for entire watershed
    'HYDROLOGICAL_MODEL': 'SUMMA',
    'ROUTING_MODEL': 'mizuRoute',
    'EXPERIMENT_TIME_START': '2004-01-01 01:00',
    'EXPERIMENT_TIME_END': '2018-12-31 23:00',
    'CALIBRATION_PERIOD': '2004-01-01, 2010-12-31',
    'EVALUATION_PERIOD': '2011-01-01, 2018-12-31',
    'SPINUP_PERIOD': '2004-01-01, 2005-12-31',
    'STATION_ID': '05BB001',                     # WSC streamflow station
    'DOWNLOAD_WSC_DATA': True
}

config_dict.update(config_updates)

# Add experiment metadata for traceability
config_dict['NOTEBOOK_CREATION_TIME'] = datetime.now().isoformat()
config_dict['NOTEBOOK_CREATOR'] = 'CONFLUENCE_Tutorial_02a'
config_dict['SCALING_TRANSITION'] = 'Point-scale to basin-scale lumped modeling'

# Save configuration
temp_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_basin_notebook.yaml'
with open(temp_config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)

print(f"✅ Configuration saved: {temp_config_path}")

# =============================================================================
# SYSTEM INITIALIZATION AND PROJECT STRUCTURE
# =============================================================================

print("\n🏗️  Initializing CONFLUENCE for Basin-Scale Modeling...")

# Initialize CONFLUENCE with basin configuration
confluence = CONFLUENCE(temp_config_path)

print(f"✅ System initialized with {len(confluence.managers)} managers")

# Create project structure
print(f"\n📁 Creating basin-scale project structure for {config_dict['DOMAIN_NAME']}...")
project_dir = confluence.managers['project'].setup_project()
pour_point_path = confluence.managers['project'].create_pour_point()

print(f"✅ Project directory: {project_dir}")
print(f"✅ Pour point created: {pour_point_path}")

# =============================================================================
# BASIN-SCALE CONFIGURATION SUMMARY
# =============================================================================

print(f"\n🏔️ Bow River at Banff Configuration Summary:")
basin_info = {
    "Watershed": "Bow River at Banff, Canadian Rockies",
    "Pour Point": f"{config_dict['POUR_POINT_COORDS']} (WSC Station {config_dict['STATION_ID']})",
    "Domain Method": f"{config_dict['DOMAIN_DEFINITION_METHOD']} (automatic watershed delineation)",
    "Discretization": f"{config_dict['DOMAIN_DISCRETIZATION']} (single HRU for entire basin)",
    "Expected Area": "~2,210 km² (from digital elevation model)",
    "Elevation Range": "1,384m (outlet) to >3,400m (headwaters)",
    "Climate": "Snow-dominated mountain system with pronounced seasonality",
    "Simulation Period": f"{config_dict['EXPERIMENT_TIME_START']} to {config_dict['EXPERIMENT_TIME_END']}"
}

for key, value in basin_info.items():
    print(f"   🌊 {key}: {value}")

print(f"\n🎯 Basin-Scale Modeling Framework:")
modeling_aspects = [
    "Watershed delineation: Automatic boundary identification from DEM",
    "Lumped representation: Spatially-averaged characteristics",
    "Streamflow routing: Surface water transport to outlet", 
    "WSC validation: Observed streamflow at gauging station",
    "Integrated response: Basin-wide water balance closure"
]

for aspect in modeling_aspects:
    print(f"   🏞️  {aspect}")

print(f"\n🔄 Workflow Status:")
workflow_status = confluence.workflow_orchestrator.get_workflow_status()
print(f"   Total steps: {workflow_status['total_steps']}")
print(f"   Completed: {workflow_status['completed_steps']}")
print(f"   Ready for watershed delineation and basin characterization")

print(f"\n🎓 Scaling Insights:")
scaling_insights = [
    "Same CONFLUENCE framework handles point-scale to basin-scale transition",
    "Lumped modeling provides basin-averaged process representation",
    "Streamflow integration captures emergent watershed behavior",
    "Foundation for distributed modeling with spatial heterogeneity",
    "Validation shifts from local states to integrated basin response"
]

for insight in scaling_insights:
    print(f"   📈 {insight}")

print(f"\n🚀 Basin-scale setup complete - Ready for watershed delineation and streamflow modeling!")

## Step 2: Basin Representation and Spatial Discretization Fundamentals
The transition from point-scale to basin-scale modeling requires fundamental decisions about how to represent spatial heterogeneity within the watershed. Unlike point-scale modeling where we assume uniform conditions, basin-scale modeling must address the challenge of capturing spatial variability while maintaining computational tractability.
Scientific Context: Basin Representation Philosophy
Spatial Heterogeneity Challenges:

- Elevation Gradients: Temperature lapse rates, precipitation patterns, snow line dynamics
- Vegetation Patterns: Forest vs alpine zones affecting evapotranspiration and interception
- Soil Variability: Infiltration, storage capacity, and drainage characteristics
- Topographic Effects: Slope, aspect, and drainage network configuration
- Climate Gradients: Orographic precipitation, temperature inversions, wind patterns

Representation Strategies:

- Lumped Approach: Single computational unit with averaged characteristics
- Semi-Distributed: Multiple units based on similarity (elevation bands, soil types, land cover)
- Fully Distributed: Grid-based representation with explicit spatial patterns

The Grouped Response Unit (GRU) concept provides flexible spatial discretization, allowing users to choose the appropriate level of complexity for their scientific objectives and computational constraints.

In [ ]:
# =============================================================================
# STEP 2: BASIN REPRESENTATION AND SPATIAL DISCRETIZATION
# =============================================================================

print("=== Step 2: Basin Representation and Spatial Discretization ===")
print("Transitioning from point-scale to integrated watershed modeling")

print(f"\n🏔️ Basin Representation Philosophy:")
representation_concepts = [
    "Spatial heterogeneity: How to capture watershed variability",
    "Computational units: Balance between complexity and tractability", 
    "Process scaling: From hillslope to watershed-scale integration",
    "Hydrologic similarity: Grouping areas with similar response",
    "Emergent behavior: Basin-scale patterns from distributed processes"
]

for concept in representation_concepts:
    print(f"   🌍 {concept}")

# Update configuration for GRU-based discretization
print(f"\n⚙️  Configuring Spatial Discretization:")
print(f"   Definition Method: {config_dict['DOMAIN_DEFINITION_METHOD']} (watershed delineation)")
print(f"   Discretization: GRUs (Grouped Response Units)")

# Update discretization method to GRUs for this demonstration
config_dict['DOMAIN_DISCRETIZATION'] = 'GRUs'

# Save updated configuration
with open(temp_config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)

# Reinitialize with updated config
confluence = CONFLUENCE(temp_config_path)

# =============================================================================
# RAPID ATTRIBUTE ACQUISITION FOR BASIN CHARACTERIZATION
# =============================================================================

print(f"\n🗺️  Acquiring Basin-Scale Geospatial Attributes...")
print(f"   Watershed: Bow River at Banff ({config_dict['POUR_POINT_COORDS']})")
print(f"   Bounding box: {config_dict.get('BOUNDING_BOX_COORDS', 'Auto-generated from watershed')}")

print(f"\n📊 Expected Mountain Watershed Attributes:")
mountain_attributes = [
    "Digital Elevation Model: MERIT DEM for watershed delineation",
    "Steep topography: High elevation gradients (1,384m to >3,400m)",
    "Alpine vegetation: Transition from montane forest to alpine zones",
    "Mountain soils: Thin soils over bedrock with variable drainage", 
    "Snow-dominated climate: Strong elevation-dependent processes"
]

for attr in mountain_attributes:
    print(f"   ⛰️  {attr}")

# Execute attribute acquisition
print(f"\n⬇️  Executing geospatial attribute acquisition...")
confluence.managers['data'].acquire_attributes()
print("✅ Basin-scale attribute acquisition complete")

# =============================================================================
# WATERSHED DELINEATION: DEFINING THE COMPUTATIONAL DOMAIN
# =============================================================================

print(f"\n🌊 Watershed Delineation: Defining Basin Boundaries...")
print(f"   Method: {config_dict['DOMAIN_DEFINITION_METHOD']} (automatic from DEM)")
print(f"   Pour point: {config_dict['POUR_POINT_COORDS']} (WSC Station {config_dict['STATION_ID']})")

print(f"\n🔧 Delineation Process:")
delineation_steps = [
    "DEM preprocessing: Fill sinks and establish flow directions",
    "Flow accumulation: Calculate upslope contributing area",
    "Stream network: Extract drainage channels above threshold",
    "Watershed boundary: Trace contributing area to pour point",
    "Quality control: Verify realistic watershed characteristics"
]

for step in delineation_steps:
    print(f"   💧 {step}")

# Execute watershed delineation
print(f"\n⚙️  Executing watershed delineation...")
watershed_path = confluence.managers['domain'].define_domain()
print("✅ Watershed delineation complete")

# =============================================================================
# DOMAIN DISCRETIZATION: CREATING COMPUTATIONAL UNITS
# =============================================================================

print(f"\n🔷 Domain Discretization: Creating Computational Units...")
print(f"   Method: {config_dict['DOMAIN_DISCRETIZATION']} (Grouped Response Units)")
print(f"   Philosophy: Balance spatial detail with computational efficiency")

print(f"\n🎯 GRU Discretization Approach:")
gru_concepts = [
    "Hydrologic similarity: Group areas with similar runoff response",
    "Topographic coherence: Maintain drainage network connectivity", 
    "Computational efficiency: Reduce model complexity while preserving key processes",
    "Parameter parsimony: Enable meaningful calibration with available data",
    "Process representation: Capture dominant basin-scale patterns"
]

for concept in gru_concepts:
    print(f"   🔸 {concept}")

# Execute domain discretization
print(f"\n🔧 Executing domain discretization...")
hru_path = confluence.managers['domain'].discretize_domain()
print("✅ Domain discretization complete")

# =============================================================================
# VISUALIZATION AND ANALYSIS OF BASIN REPRESENTATION
# =============================================================================

print(f"\n📊 Analyzing Created Basin Representation...")

# Verify and visualize watershed and HRUs
if watershed_path and watershed_path.exists() and hru_path and hru_path.exists():
    
    # Load spatial data
    watershed_gdf = gpd.read_file(watershed_path)
    hru_gdf = gpd.read_file(hru_path)
    pour_point_gdf = gpd.read_file(pour_point_path)
    
    print(f"\n📋 Basin Characteristics:")
    total_area_m2 = watershed_gdf.geometry.area.sum()
    total_area_km2 = total_area_m2 / 1e6
    print(f"   Watershed area: {total_area_km2:.1f} km²")
    print(f"   Number of GRUs: {len(hru_gdf)}")
    print(f"   Average GRU size: {total_area_km2/len(hru_gdf):.1f} km²")
    
    # Display HRU characteristics if available
    if 'elevation' in hru_gdf.columns:
        print(f"   Elevation range: {hru_gdf['elevation'].min():.0f}m to {hru_gdf['elevation'].max():.0f}m")
    if 'slope' in hru_gdf.columns:
        print(f"   Slope range: {hru_gdf['slope'].min():.1f}° to {hru_gdf['slope'].max():.1f}°")
    
    # Create comprehensive visualization
    print(f"\n🗺️  Creating basin representation visualization...")
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Left plot: Watershed boundary and pour point
    ax1 = axes[0]
    watershed_gdf.plot(ax=ax1, facecolor='lightblue', edgecolor='navy', 
                      linewidth=2, alpha=0.7)
    pour_point_gdf.plot(ax=ax1, color='red', markersize=100, marker='o',
                       edgecolor='white', linewidth=2, zorder=5)
    
    ax1.set_title('Delineated Watershed Boundary', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Longitude', fontsize=12)
    ax1.set_ylabel('Latitude', fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Add area annotation
    ax1.text(0.02, 0.98, f'Area: {total_area_km2:.1f} km²\nPour Point: WSC {config_dict["STATION_ID"]}',
             transform=ax1.transAxes, fontsize=10, verticalalignment='top',
             bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.3'))
    
    # Right plot: HRU discretization
    ax2 = axes[1]
    
    # Color HRUs by elevation if available, otherwise by area
    if 'elevation' in hru_gdf.columns:
        hru_gdf.plot(ax=ax2, column='elevation', cmap='terrain', 
                    edgecolor='black', linewidth=0.5, legend=True)
        colorbar_label = 'Elevation (m)'
    else:
        hru_gdf.plot(ax=ax2, column=hru_gdf.geometry.area, cmap='viridis',
                    edgecolor='black', linewidth=0.5, legend=True) 
        colorbar_label = 'Area (deg²)'
    
    pour_point_gdf.plot(ax=ax2, color='red', markersize=100, marker='o',
                       edgecolor='white', linewidth=2, zorder=5)
    
    ax2.set_title(f'GRU Discretization ({len(hru_gdf)} units)', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Longitude', fontsize=12)
    ax2.set_ylabel('Latitude', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # Add discretization info
    ax2.text(0.02, 0.98, f'GRUs: {len(hru_gdf)}\nAvg. size: {total_area_km2/len(hru_gdf):.1f} km²',
             transform=ax2.transAxes, fontsize=10, verticalalignment='top',
             bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.3'))
    
    plt.suptitle(f'Bow River at Banff: Basin Representation', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Summary table of GRU characteristics
    if len(hru_gdf) <= 10:  # Show details for small number of GRUs
        print(f"\n📋 Individual GRU Characteristics:")
        for idx, gru in hru_gdf.iterrows():
            gru_area = gru.geometry.area / 1e6  # Convert to km²
            print(f"   GRU {gru.get('GRU_ID', idx+1)}: {gru_area:.1f} km²", end="")
            if 'elevation' in gru:
                print(f", {gru['elevation']:.0f}m elevation", end="")
            if 'landclass' in gru:
                print(f", {gru['landclass']}")
            else:
                print()

else:
    print("⚠️  Cannot visualize basin representation - check delineation outputs")

# =============================================================================
# BASIN REPRESENTATION SUMMARY AND EXPERIMENTAL OPPORTUNITIES  
# =============================================================================

print(f"\n🎯 Basin Representation Summary:")
representation_summary = [
    f"✅ Watershed successfully delineated ({total_area_km2:.1f} km²)",
    f"✅ {len(hru_gdf)} GRUs created using {config_dict['DOMAIN_DISCRETIZATION']} method",
    "✅ Spatial discretization balances complexity with computational efficiency",
    "✅ Foundation established for lumped basin modeling",
    "✅ Framework ready for alternative discretization experiments"
]

for summary in representation_summary:
    print(f"   {summary}")

print(f"\n🔬 Scientific Insights:")
scientific_insights = [
    "GRU approach captures essential spatial variability",
    "Watershed delineation provides physically-based domain boundary", 
    "Spatial aggregation enables computationally-efficient modeling",
    "Foundation established for process-based streamflow simulation",
    "Balance achieved between spatial detail and parameter identifiability"
]

for insight in scientific_insights:
    print(f"   🧠 {insight}")

print(f"\n🧪 Experimental Opportunity: Alternative Discretization Methods")
print(f"   Current setup: DOMAIN_DISCRETIZATION = '{config_dict['DOMAIN_DISCRETIZATION']}'")
print(f"   
Alternative methods to explore:")

alternative_methods = [
    "'elevation' - Create elevation bands for temperature/snow gradients",
    "'landclass' - Discretize by vegetation/land cover types",
    "'soilclass' - Group by soil hydraulic properties", 
    "'radiation' - Organize by solar radiation patterns",
    "'lumped' - Single HRU representing entire watershed"
]

for method in alternative_methods:
    print(f"      🔄 {method}")

print(f"\n💡 To Experiment:")
print(f"   1. Change DOMAIN_DISCRETIZATION in configuration")
print(f"   2. Re-run domain discretization step")  
print(f"   3. Compare computational units and model performance")
print(f"   4. Analyze trade-offs between complexity and accuracy")

print(f"\n🚀 Basin representation complete - Ready for data preprocessing and model execution!")
print(f"   → Watershed: Physically-based boundary from DEM")
print(f"   → GRUs: Balanced spatial representation") 
print(f"   → Framework: Extensible to alternative discretization strategies")
print(f"   → Next: Model-agnostic preprocessing and streamflow simulation")

## Step 3: Streamlined Data Pipeline for Basin-Scale Streamflow Modeling
The same model-agnostic preprocessing framework now scales from point validation to basin-scale streamflow simulation. The core philosophy remains unchanged—standardized, quality-controlled data products—but the spatial context shifts from single locations to integrated watershed responses.
Data Pipeline Scaling: Point → Basin

- Forcing Data: Same ERA5 global data, now basin-averaged across watershed
- Validation Target: Streamflow hydrographs vs local states (SWE, SM, LE)
- Spatial Processing: Watershed-scale remapping vs single-point extraction
- Temporal Integration: Daily streamflow vs sub-daily energy cycles
- Process Focus: Integrated water balance with routing vs isolated vertical processes

The same CONFLUENCE preprocessing pipeline handles both scales seamlessly, demonstrating the framework's scalability while maintaining data quality and reproducibility standards.

In [ ]:
# =============================================================================
# STEP 3: STREAMLINED DATA PIPELINE FOR BASIN-SCALE STREAMFLOW MODELING
# =============================================================================

print("=== Step 3: Basin-Scale Data Pipeline for Streamflow Simulation ===")
print("Scaling model-agnostic preprocessing from point validation to watershed integration")

# =============================================================================
# STREAMFLOW OBSERVATIONS: WSC HYDROMETRIC DATA
# =============================================================================

print(f"\n🌊 Processing Streamflow Observations for Basin Outlet...")
print(f"   Station: WSC {config_dict['STATION_ID']} (Bow River at Banff)")
print(f"   Data source: Water Survey of Canada (HYDAT database)")
print(f"   Validation target: Daily streamflow hydrograph")

print(f"\n🎯 Streamflow Validation Framework:")
streamflow_context = [
    "Integrated basin response: All upstream processes contribute to outlet flow",
    "Daily resolution: Captures seasonal cycles and flood events",
    "Long-term records: Multi-decadal observations for robust evaluation",
    "Hydrologic signatures: Peak flows, base flows, seasonal timing",
    "Water balance closure: Basin-scale precipitation → streamflow relationship"
]

for context in streamflow_context:
    print(f"   📊 {context}")

# Execute streamflow data processing
print(f"\n📥 Processing WSC streamflow observations...")
confluence.managers['data'].process_observed_data()
print("✅ Streamflow data processing complete")

print(f"\n🔬 Scientific Value of Streamflow Validation:")
validation_benefits = [
    "Direct measurement of integrated watershed response",
    "Natural integration of all upstream hydrological processes",
    "Objective function for basin-scale model calibration",
    "Benchmark for distributed vs lumped modeling approaches",
    "Foundation for water resources management applications"
]

for benefit in validation_benefits:
    print(f"   🌊 {benefit}")

# =============================================================================
# BASIN-AVERAGED METEOROLOGICAL FORCING
# =============================================================================

print(f"\n🌦️  Acquiring Basin-Averaged Meteorological Forcing...")
print(f"   Watershed area: ~{total_area_km2:.1f} km² (Bow River at Banff)")
print(f"   Elevation range: {hru_gdf['elevation'].min():.0f}m to {hru_gdf['elevation'].max():.0f}m")
print(f"   Climate context: Snow-dominated mountain watershed")

print(f"\n📈 Basin-Scale Forcing Considerations:")
forcing_considerations = [
    "Orographic precipitation: Enhanced snowfall at high elevations",
    "Temperature gradients: Lapse rate effects across elevation zones", 
    "Spatial averaging: ERA5 grid cells → watershed-representative values",
    "Seasonal patterns: Distinct snow accumulation and melt periods",
    "Extreme events: Atmospheric rivers and rain-on-snow episodes"
]

for consideration in forcing_considerations:
    print(f"   ⛰️  {consideration}")

# Execute forcing acquisition (commented for demonstration)
print(f"\n⬇️  Executing basin-scale forcing acquisition...")
# confluence.managers['data'].acquire_forcings()
print("✅ ERA5 forcing acquisition complete (simulated)")

# =============================================================================
# MODEL-AGNOSTIC PREPROCESSING: BASIN-SCALE SPATIAL PROCESSING
# =============================================================================

print(f"\n🔧 Model-Agnostic Preprocessing for Basin-Scale Integration...")

print(f"\n⚙️  Basin-Scale Preprocessing Components:")
preprocessing_components = [
    "Spatial remapping: ERA5 grids → watershed-averaged forcing",
    "GRU characterization: Zonal statistics for each computational unit",
    "Elevation processing: Lapse rate corrections for mountain gradients",
    "Quality control: Gap filling and temporal consistency checks",
    "Format standardization: Model-independent NetCDF outputs"
]

for component in preprocessing_components:
    print(f"   🔄 {component}")

# Execute model-agnostic preprocessing
print(f"\n⚙️  Executing basin-scale model-agnostic preprocessing...")
confluence.managers['data'].run_model_agnostic_preprocessing()
print("✅ Model-agnostic preprocessing complete")

print(f"\n🎯 Basin-Scale Preprocessing Outputs:")
basin_outputs = [
    f"Watershed-averaged forcing: {len(hru_gdf)} GRUs with representative meteorology",
    "GRU attribute table: Elevation, slope, soil, land cover characteristics",
    "Spatial mapping files: Conservative remapping for water/energy balance",
    "Quality reports: Basin-scale data coverage and uncertainty assessment"
]

for output in basin_outputs:
    print(f"   📦 {output}")

# =============================================================================
# MODEL-SPECIFIC PREPROCESSING: SUMMA + MIZUROUTE CONFIGURATION
# =============================================================================

print(f"\n🌊 SUMMA + mizuRoute Configuration for Basin Streamflow...")
print(f"   Hydrological model: {config_dict['HYDROLOGICAL_MODEL']} (process-based)")
print(f"   Routing model: {config_dict['ROUTING_MODEL']} (streamflow routing)")
print(f"   Integration: Vertical water balance + horizontal flow routing")

print(f"\n🔧 Basin-Scale Model Configuration:")
model_config = [
    "SUMMA setup: Process-based energy/water balance for each GRU",
    "Parameter assignment: Basin-appropriate soil, vegetation, snow parameters",
    "Initial conditions: Realistic starting states for mountain watershed",
    "mizuRoute coupling: GRU runoff → stream network → outlet streamflow",
    "Output configuration: Daily streamflow and water balance components"
]

for config in model_config:
    print(f"   🌲 {config}")

# Execute model-specific preprocessing
print(f"\n🔧 Executing SUMMA + mizuRoute preprocessing...")
confluence.managers['model'].preprocess_models()
print("✅ Basin-scale model configuration complete")

print(f"\n📊 Expected Basin-Scale Outputs:")
model_outputs = [
    "Daily streamflow at basin outlet (m³/s)",
    "GRU-level water balance components", 
    "Snow accumulation and melt dynamics",
    "Soil moisture and evapotranspiration",
    "Streamflow routing through channel network"
]

for output in model_outputs:
    print(f"   📈 {output}")

# =============================================================================
# DATA PIPELINE SUMMARY FOR BASIN-SCALE MODELING
# =============================================================================

print(f"\n✅ Basin-Scale Data Pipeline Summary:")

pipeline_achievements = [
    "✅ WSC streamflow observations processed for validation",
    "✅ Basin-averaged ERA5 forcing acquired and quality-controlled",
    "✅ Model-agnostic preprocessing creates standardized watershed products",
    "✅ SUMMA + mizuRoute configured for integrated basin simulation",
    "✅ Streamflow routing framework ready for outlet validation"
]

for achievement in pipeline_achievements:
    print(f"   {achievement}")

print(f"\n🔬 Scaling Benefits Demonstrated:")
scaling_benefits = [
    "Same preprocessing framework scales from point to basin applications",
    "Consistent data quality standards maintained across spatial scales",
    "Model-agnostic approach enables multi-model basin comparisons",
    "Standardized outputs support automated evaluation and benchmarking",
    "Reproducible workflow facilitates collaborative watershed research"
]

for benefit in scaling_benefits:
    print(f"   🎯 {benefit}")

print(f"\n🌐 Framework Versatility Across Scales:")
print(f"   📊 Same preprocessing pipeline handles:")
print(f"      • Tutorial 01a: Point-scale snow/soil validation")
print(f"      • Tutorial 01b: Point-scale energy flux validation")
print(f"      • Tutorial 02a: Basin-scale streamflow simulation") 
print(f"      • Future: Large-sample hydrology across thousands of basins")

print(f"\n🚀 Ready for basin-scale SUMMA + mizuRoute execution!")
print(f"   → Preprocessed inputs: Watershed-scale and quality-controlled")
print(f"   → Model configuration: Integrated vertical and horizontal processes")
print(f"   → Validation target: WSC streamflow observations prepared")
print(f"   → Next step: Basin-scale simulation and streamflow evaluation")

## Step 4: Streamlined Basin-Scale Model Execution
The same SUMMA process-based physics now scales from point validation to integrated basin simulation, but with the critical addition of streamflow routing. This represents a fundamental modeling advancement: from isolated vertical processes to coupled vertical-horizontal water transport that generates streamflow at the basin outlet.
Model Execution Scaling: Point → Basin

- Spatial Integration: Single HRU → Multiple GRUs with routing connectivity
- Process Coupling: Vertical water balance → Vertical + horizontal flow routing
- Output Target: Local states → Streamflow hydrograph at outlet
- Temporal Integration: Sub-daily energy cycles → Daily streamflow generation
- Validation Shift: Direct state comparison → Integrated basin response

The same workflow orchestration ensures robust execution while mizuRoute routing transforms distributed runoff into the streamflow observations that drive water resources management.

In [ ]:
# =============================================================================
# STEP 4: STREAMLINED BASIN-SCALE MODEL EXECUTION
# =============================================================================

print("=== Step 4: Basin-Scale SUMMA + mizuRoute Execution ===")
print("Integrating process-based physics with streamflow routing for watershed simulation")

# =============================================================================
# INTEGRATED BASIN SIMULATION: SUMMA + MIZUROUTE
# =============================================================================

print(f"\n🌊 Executing Integrated Basin-Scale Simulation...")
print(f"   Hydrological model: {config_dict['HYDROLOGICAL_MODEL']} (process-based physics)")
print(f"   Routing model: {config_dict['ROUTING_MODEL']} (streamflow routing)")
print(f"   Domain: {config_dict['DOMAIN_NAME']} ({len(hru_gdf)} GRUs, {total_area_km2:.1f} km²)")
print(f"   Target: Streamflow at WSC {config_dict['STATION_ID']}")

print(f"\n⚡ Integrated Modeling Framework:")
integrated_processes = [
    "SUMMA GRU simulation: Process-based water/energy balance for each unit",
    "Runoff generation: Surface and subsurface flow from each GRU",
    "mizuRoute routing: Channel flow transport through stream network",
    "Outlet integration: Basin-wide runoff → streamflow hydrograph",
    "Temporal coupling: Hourly physics → daily streamflow aggregation"
]

for process in integrated_processes:
    print(f"   🌊 {process}")

# Execute the integrated model system
print(f"\n🏃‍♂️ Running SUMMA + mizuRoute basin simulation...")
confluence.managers['model'].run_models()
print("✅ Basin-scale integrated simulation complete")

# =============================================================================
# QUICK VERIFICATION AND STREAMFLOW OUTPUT
# =============================================================================

print(f"\n🔍 Basin Simulation Output Verification...")

# Locate and verify simulation outputs
sim_dir = confluence.project_dir / "simulations" / config_dict['EXPERIMENT_ID']
summa_outputs = sim_dir / "SUMMA"
routing_outputs = sim_dir / "mizuRoute"

print(f"   📁 SUMMA outputs: {summa_outputs}")
print(f"   📁 mizuRoute outputs: {routing_outputs}")

# Check for key output files
key_outputs = {
    "SUMMA daily": f"{config_dict['EXPERIMENT_ID']}_day.nc",
    "mizuRoute streamflow": f"{config_dict['EXPERIMENT_ID']}_mizuRoute_output.nc"
}

for output_type, filename in key_outputs.items():
    summa_file = summa_outputs / filename
    routing_file = routing_outputs / filename
    
    if summa_file.exists():
        file_size = summa_file.stat().st_size / (1024*1024)  # MB
        print(f"   ✅ {output_type}: {filename} ({file_size:.1f} MB)")
    elif routing_file.exists():
        file_size = routing_file.stat().st_size / (1024*1024)  # MB  
        print(f"   ✅ {output_type}: {filename} ({file_size:.1f} MB)")
    else:
        print(f"   📋 {output_type}: {filename} (checking...)")

print(f"\n📊 Basin-Scale Simulation Products:")
simulation_products = [
    "Streamflow hydrograph: Daily discharge at basin outlet",
    "Water balance components: ET, storage changes, routing fluxes",
    "Spatial patterns: GRU-level runoff generation", 
    "Temporal dynamics: Seasonal cycles and event responses",
    "Quality metrics: Mass balance closure and physical consistency"
]

for product in simulation_products:
    print(f"   📈 {product}")

# =============================================================================
# STREAMFLOW GENERATION VERIFICATION
# =============================================================================

print(f"\n🌊 Streamflow Generation Assessment...")

# Quick check of streamflow outputs
try:
    # Look for mizuRoute output file
    routing_files = list(routing_outputs.glob("*.nc"))
    if routing_files:
        import xarray as xr
        
        # Load first routing output file
        routing_ds = xr.open_dataset(routing_files[0])
        
        print(f"   ✅ Routing simulation loaded")
        print(f"   Variables: {list(routing_ds.data_vars)}")
        
        # Check for streamflow variable
        if 'IRFroutedRunoff' in routing_ds.data_vars:
            streamflow = routing_ds['IRFroutedRunoff']
            print(f"   📊 Streamflow range: {float(streamflow.min()):.2f} to {float(streamflow.max()):.2f} m³/s")
            print(f"   📅 Simulation period: {streamflow.time.min().values} to {streamflow.time.max().values}")
        
        routing_ds.close()
        
except Exception as e:
    print(f"   📋 Streamflow verification pending: {e}")

print(f"\n🎯 Basin-Scale Integration Achievements:")
integration_achievements = [
    "✅ Multi-GRU SUMMA simulation executed successfully",
    "✅ Runoff routing through stream network completed",
    "✅ Streamflow hydrograph generated at basin outlet",
    "✅ Integrated water balance maintained across spatial scales",
    "✅ Foundation established for streamflow validation"
]

for achievement in integration_achievements:
    print(f"   {achievement}")

# =============================================================================
# SCIENTIFIC INTEGRATION SUMMARY
# =============================================================================

print(f"\n🔬 Scientific Integration Summary:")

print(f"\n🌊 Process Integration Accomplished:")
process_integration = [
    "Vertical physics: Energy/water balance at GRU scale",
    "Horizontal routing: Streamflow transport through channels",
    "Spatial aggregation: Multiple GRUs → single outlet response",
    "Temporal integration: Sub-daily processes → daily streamflow",
    "Scale coupling: Hillslope runoff → watershed streamflow"
]

for integration in process_integration:
    print(f"   ⚙️  {integration}")

print(f"\n🎓 Modeling Advances Demonstrated:")
modeling_advances = [
    "Same process-based physics scales from point to basin applications",
    "Routing integration enables streamflow prediction capability", 
    "Distributed runoff generation maintains spatial process detail",
    "Quality-assured execution ensures physical realism",
    "Reproducible workflow supports operational applications"
]

for advance in modeling_advances:
    print(f"   📈 {advance}")

print(f"\n✨ Key Scientific Achievement:")
print(f"   🌊 Successfully transitioned from point-scale process validation")
print(f"      to integrated basin-scale streamflow simulation")
print(f"   🔄 Same CONFLUENCE framework handles both scales seamlessly")
print(f"   🎯 Ready for comprehensive streamflow evaluation and validation")

print(f"\n🚀 Basin-scale simulation complete - Ready for streamflow evaluation!")
print(f"   → Integrated modeling: SUMMA physics + mizuRoute routing")
print(f"   → Streamflow generation: Basin outlet hydrograph available")
print(f"   → Validation target: WSC observations for performance assessment")
print(f"   → Scientific foundation: Process-based watershed modeling achieved")

## Step 5: Streamflow Evaluation and Basin Performance Assessment
The same CONFLUENCE evaluation framework now transitions from point-scale validation to basin-scale streamflow assessment. This represents a fundamental shift in validation philosophy: from direct process comparison (SWE, SM, LE) to integrated response evaluation where all upstream processes collectively generate the streamflow signal at the basin outlet.
Evaluation Framework Transition: Point → Basin

- Validation Target: Local states (SWE, soil moisture, energy fluxes) → Streamflow hydrograph
- Process Integration: Direct measurement comparison → Emergent watershed response
- Temporal Patterns: Sub-daily cycles → Seasonal flow regimes and flood events
- Performance Metrics: State variable accuracy → Hydrologic signatures and timing
- Scientific Interpretation: Process physics → Water balance closure and prediction skill

The same evaluation infrastructure seamlessly handles this transition, demonstrating CONFLUENCE's versatility across validation scales while maintaining rigorous performance assessment standards.

In [ ]:
# =============================================================================
# STEP 5: STREAMLINED STREAMFLOW EVALUATION AND BASIN PERFORMANCE ASSESSMENT
# =============================================================================

print("=== Step 5: Basin-Scale Streamflow Evaluation ===")
print("Comprehensive assessment of integrated watershed response and prediction skill")

# =============================================================================
# STREAMFLOW DATA LOADING AND INTEGRATION
# =============================================================================

print(f"\n🌊 Loading Streamflow Simulation and Observations...")

# Load observed streamflow data
obs_path = confluence.project_dir / "observations" / "streamflow" / "preprocessed" / f"{config_dict['DOMAIN_NAME']}_streamflow_processed.csv"

if obs_path.exists():
    obs_df = pd.read_csv(obs_path, parse_dates=['datetime'])
    obs_df.set_index('datetime', inplace=True)
    
    print(f"✅ WSC observations loaded")
    print(f"   Station: {config_dict['STATION_ID']} (Bow River at Banff)")
    print(f"   Period: {obs_df.index.min()} to {obs_df.index.max()}")
    print(f"   Flow range: {obs_df['discharge_cms'].min():.1f} to {obs_df['discharge_cms'].max():.1f} m³/s")
else:
    print(f"⚠️  Observed streamflow not found at {obs_path}")
    obs_df = None

# Load simulated streamflow from mizuRoute
routing_dir = confluence.project_dir / "simulations" / config_dict['EXPERIMENT_ID'] / "mizuRoute"
routing_files = list(routing_dir.glob("*.nc"))

if routing_files:
    # Load mizuRoute output
    routing_ds = xr.open_dataset(routing_files[0])
    
    # Extract streamflow variable (typically IRFroutedRunoff)
    if 'IRFroutedRunoff' in routing_ds.data_vars:
        sim_streamflow = routing_ds['IRFroutedRunoff']
        
        # Convert to pandas for easier analysis
        sim_df = sim_streamflow.to_pandas()
        
        print(f"✅ mizuRoute simulation loaded")
        print(f"   Period: {sim_df.index.min()} to {sim_df.index.max()}")
        print(f"   Flow range: {sim_df.min():.1f} to {sim_df.max():.1f} m³/s")
        
        routing_ds.close()
    else:
        print(f"⚠️  Streamflow variable not found in mizuRoute output")
        print(f"   Available variables: {list(routing_ds.data_vars)}")
        sim_df = None
        routing_ds.close()
else:
    print(f"⚠️  mizuRoute output files not found in {routing_dir}")
    sim_df = None

# =============================================================================
# STREAMFLOW PERFORMANCE EVALUATION
# =============================================================================

if obs_df is not None and sim_df is not None:
    print(f"\n📊 Streamflow Performance Assessment...")
    
    # Align data to common period
    start_date = max(obs_df.index.min(), sim_df.index.min())
    end_date = min(obs_df.index.max(), sim_df.index.max())
    
    # Skip initial spinup period
    start_date = start_date + pd.DateOffset(months=6)
    
    print(f"   Evaluation period: {start_date} to {end_date}")
    print(f"   Duration: {(end_date - start_date).days} days")
    
    # Filter to common period and resample to daily
    obs_daily = obs_df['discharge_cms'].resample('D').mean().loc[start_date:end_date]
    sim_daily = sim_df.resample('D').mean().loc[start_date:end_date]
    
    # Remove any remaining NaN values
    valid_mask = ~(obs_daily.isna() | sim_daily.isna())
    obs_valid = obs_daily[valid_mask]
    sim_valid = sim_daily[valid_mask]
    
    print(f"   Valid paired observations: {len(obs_valid)} days")
    
    # Calculate comprehensive performance metrics
    print(f"\n📈 Streamflow Performance Metrics:")
    
    # Basic statistics
    rmse = np.sqrt(((obs_valid - sim_valid) ** 2).mean())
    bias = (sim_valid - obs_valid).mean()
    mae = np.abs(obs_valid - sim_valid).mean()
    
    # Relative metrics
    pbias = 100 * bias / obs_valid.mean()
    
    # Nash-Sutcliffe Efficiency
    nse = 1 - ((obs_valid - sim_valid) ** 2).sum() / ((obs_valid - obs_valid.mean()) ** 2).sum()
    
    # Kling-Gupta Efficiency  
    r = obs_valid.corr(sim_valid)
    alpha = sim_valid.std() / obs_valid.std()
    beta = sim_valid.mean() / obs_valid.mean()
    kge = 1 - np.sqrt((r - 1)**2 + (alpha - 1)**2 + (beta - 1)**2)
    
    # Display performance metrics
    print(f"   📊 RMSE: {rmse:.2f} m³/s")
    print(f"   📊 Bias: {bias:+.2f} m³/s ({pbias:+.1f}%)")
    print(f"   📊 MAE: {mae:.2f} m³/s")
    print(f"   📊 Correlation (r): {r:.3f}")
    print(f"   📊 Nash-Sutcliffe (NSE): {nse:.3f}")
    print(f"   📊 Kling-Gupta (KGE): {kge:.3f}")
    
    # Hydrologic signature analysis
    print(f"\n🌊 Hydrologic Signature Analysis:")
    
    # Flow statistics
    obs_q95 = obs_valid.quantile(0.95)  # High flows
    sim_q95 = sim_valid.quantile(0.95)
    obs_q05 = obs_valid.quantile(0.05)  # Low flows  
    sim_q05 = sim_valid.quantile(0.05)
    
    print(f"   High flows (Q95): Obs={obs_q95:.1f}, Sim={sim_q95:.1f} m³/s")
    print(f"   Low flows (Q05): Obs={obs_q05:.1f}, Sim={sim_q05:.1f} m³/s")
    
    # Seasonal timing
    obs_monthly = obs_valid.groupby(obs_valid.index.month).mean()
    sim_monthly = sim_valid.groupby(sim_valid.index.month).mean()
    peak_month_obs = obs_monthly.idxmax()
    peak_month_sim = sim_monthly.idxmax()
    
    print(f"   Peak flow timing: Obs=Month {peak_month_obs}, Sim=Month {peak_month_sim}")
    
    # =============================================================================
    # COMPREHENSIVE STREAMFLOW VISUALIZATION
    # =============================================================================
    
    print(f"\n📈 Creating comprehensive streamflow evaluation...")
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Time series comparison (top left)
    ax1 = axes[0, 0]
    ax1.plot(obs_valid.index, obs_valid.values, 'b-', 
             label='WSC Observed', linewidth=1.5, alpha=0.8)
    ax1.plot(sim_valid.index, sim_valid.values, 'r-', 
             label='SUMMA + mizuRoute', linewidth=1.5, alpha=0.8)
    
    ax1.set_ylabel('Discharge (m³/s)', fontsize=11)
    ax1.set_title('Streamflow Time Series', fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add performance metrics
    metrics_text = f'NSE: {nse:.3f}\nKGE: {kge:.3f}\nBias: {pbias:+.1f}%'
    ax1.text(0.02, 0.95, metrics_text, transform=ax1.transAxes,
             bbox=dict(facecolor='white', alpha=0.8), fontsize=10, verticalalignment='top')
    
    # Scatter plot (top right)
    ax2 = axes[0, 1]
    ax2.scatter(obs_valid, sim_valid, alpha=0.5, c='blue', s=20)
    max_val = max(obs_valid.max(), sim_valid.max())
    ax2.plot([0, max_val], [0, max_val], 'k--', label='1:1 line')
    ax2.set_xlabel('Observed (m³/s)', fontsize=11)
    ax2.set_ylabel('Simulated (m³/s)', fontsize=11)
    ax2.set_title('Obs vs Sim Streamflow', fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Monthly climatology (bottom left)
    ax3 = axes[1, 0]
    months = range(1, 13)
    month_names = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']
    
    ax3.plot(months, obs_monthly.values, 'o-', label='Observed', 
             color='blue', linewidth=2, markersize=6)
    ax3.plot(months, sim_monthly.values, 'o-', label='Simulated', 
             color='red', linewidth=2, markersize=6)
    
    ax3.set_xticks(months)
    ax3.set_xticklabels(month_names)
    ax3.set_ylabel('Mean Discharge (m³/s)', fontsize=11)
    ax3.set_title('Seasonal Flow Regime', fontweight='bold')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Flow duration curve (bottom right)
    ax4 = axes[1, 1]
    
    # Calculate exceedance probabilities
    obs_sorted = obs_valid.sort_values(ascending=False)
    sim_sorted = sim_valid.sort_values(ascending=False)
    obs_ranks = np.arange(1., len(obs_sorted) + 1) / len(obs_sorted) * 100
    sim_ranks = np.arange(1., len(sim_sorted) + 1) / len(sim_sorted) * 100
    
    ax4.semilogy(obs_ranks, obs_sorted, 'b-', label='Observed', linewidth=2)
    ax4.semilogy(sim_ranks, sim_sorted, 'r-', label='Simulated', linewidth=2)
    
    ax4.set_xlabel('Exceedance Probability (%)', fontsize=11)
    ax4.set_ylabel('Discharge (m³/s)', fontsize=11)
    ax4.set_title('Flow Duration Curve', fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.suptitle(f'Basin-Scale Streamflow Evaluation - {config_dict["DOMAIN_NAME"]}',
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

else:
    print("⚠️  Cannot perform streamflow evaluation - missing simulation or observation data")
